In [22]:
%pip install google-colab-selenium

In [23]:
import google_colab_selenium as gs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
import os
from google.colab import drive

## **PARTE 1 (SEPTUD Y TEST DE ACCESO A LA PAGINA WEB)**

In [27]:
# CONFIGURACIÓN INICIAL
def configurar_driver():
    """
    Configura el driver de Chrome con las opciones necesarias para Colab
    """
    print("🔧 Configurando el navegador Chrome...")

    chrome_options = Options()
    # Opciones esenciales para Colab
    chrome_options.add_argument("--headless")  # Sin interfaz gráfica
    chrome_options.add_argument("--no-sandbox")  # Requerido en Colab
    chrome_options.add_argument("--disable-dev-shm-usage")  # Evita problemas de memoria
    chrome_options.add_argument("--disable-gpu")  # Desactiva GPU para mayor estabilidad
    chrome_options.add_argument("--window-size=1920,1080")  # Tamaño de ventana estándar

    # Configuración para descargas automáticas
    prefs = {
        "download.default_directory": "/content/drive/MyDrive/IDEAM_Datos",
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    chrome_options.add_experimental_option("prefs", prefs)

    # Crear el driver
    driver = gs.Chrome(options=chrome_options)
    driver.set_page_load_timeout(30)  # Timeout de 30 segundos para cargar páginas

    print("✅ Navegador configurado correctamente")
    return driver

In [49]:
def montar_google_drive():
    """
    Monta Google Drive y crea la carpeta para los datos del IDEAM
    """
    print("📂 Montando Google Drive...")
    drive.mount('/content/drive')

    # Crear carpeta para los datos del IDEAM
    carpeta_datos = '/content/IDEAM_Datos'
    os.makedirs(carpeta_datos, exist_ok=True)
    print(f"✅ Carpeta creada: {carpeta_datos}")

    return carpeta_datos

In [45]:
# FUNCIÓN PARA MANEJAR LA VENTANA DE TÉRMINOS Y CONDICIONES
def aceptar_terminos_condiciones(driver):
    """
    Maneja la ventana emergente de términos y condiciones del IDEAM
    Esta es la parte crítica que estaba fallando
    """
    print("📋 Manejando términos y condiciones...")

    try:
        # Esperar a que aparezca la ventana modal
        print("⏳ Esperando ventana de términos...")
        modal = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "dijitDialog"))
        )
        print("✅ Ventana de términos detectada")

        # Estrategia múltiple para encontrar y hacer click en el checkbox
        checkbox_encontrado = False

        # Estrategia 1: Por clase CSS específica
        try:
            print("🔍 Intentando localizar checkbox - Estrategia 1...")
            checkbox = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.checkbox.jimu-float-leading'))
            )
            checkbox.click()
            print("✅ Checkbox marcado (Estrategia 1)")
            checkbox_encontrado = True
        except:
            print("❌ Estrategia 1 falló")

        # Estrategia 2: Por input type checkbox
        if not checkbox_encontrado:
            try:
                print("🔍 Intentando localizar checkbox - Estrategia 2...")
                checkbox = driver.find_element(By.CSS_SELECTOR, 'input[type="checkbox"]')
                driver.execute_script("arguments[0].click();", checkbox)
                print("✅ Checkbox marcado (Estrategia 2)")
                checkbox_encontrado = True
            except:
                print("❌ Estrategia 2 falló")

        # Estrategia 3: Buscar por texto o etiqueta
        if not checkbox_encontrado:
            try:
                print("🔍 Intentando localizar checkbox - Estrategia 3...")
                # Buscar elemento que contenga texto relacionado con aceptación
                checkbox = driver.find_element(By.XPATH, "//div[contains(@class, 'checkbox')]")
                driver.execute_script("arguments[0].click();", checkbox)
                print("✅ Checkbox marcado (Estrategia 3)")
                checkbox_encontrado = True
            except:
                print("❌ Estrategia 3 falló")

        if not checkbox_encontrado:
            raise Exception("No se pudo encontrar el checkbox de términos y condiciones")

        # Pausa breve para asegurar que el checkbox se registre
        time.sleep(2)

        # Ahora buscar y hacer click en el botón "Aceptar"
        print("🔍 Buscando botón Aceptar...")

        # Estrategias múltiples para el botón Aceptar
        boton_encontrado = False

        # Estrategia 1: Por nombre
        try:
            boton = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[title="Aceptar"]'))
            )
            boton.click()
            print("✅ Botón Aceptar clickeado (por título)")
            boton_encontrado = True
        except:
            print("❌ Botón por nombre no encontrado")

        # Estrategia 2: Por texto del botón
        if not boton_encontrado:
            try:
                boton = driver.find_element(By.XPATH, '//div[data-dojo-attach-event="onclick:onOkClick"]')
                boton.click()
                print("✅ Botón Aceptar clickeado (por texto)")
                boton_encontrado = True
            except:
                print("❌ Botón por texto no encontrado")

        # Estrategia 3: Por clase de botón dijit
        if not boton_encontrado:
            try:
                boton = driver.find_element(By.CSS_SELECTOR, 'div[class="jimu-btn jimu-float-trailing enable-btn"]')
                if 'aceptar' in boton.text.lower():
                    boton.click()
                    print("✅ Botón Aceptar clickeado (por clase dijit)")
                    boton_encontrado = True
            except:
                print("❌ Botón por clase dijit no encontrado")

        if not boton_encontrado:
            raise Exception("No se pudo encontrar el botón Aceptar")

        # Esperar a que la ventana se cierre
        print("⏳ Esperando que se cierre la ventana...")
        WebDriverWait(driver, 15).until(
            EC.invisibility_of_element((By.CLASS_NAME, "dijitDialog"))
        )

        print("✅ Términos y condiciones aceptados correctamente")
        return True

    except TimeoutException:
        print("⚠️ Timeout: La ventana de términos no apareció o no se pudo cerrar")
        return False
    except Exception as e:
        print(f"❌ Error al manejar términos y condiciones: {str(e)}")
        return False


In [46]:
# FUNCIÓN PRINCIPAL DE NAVEGACIÓN
def navegar_a_ideam(driver):
    """
    Navega al sitio del IDEAM y maneja la página inicial
    """
    print("🌐 Navegando al sitio del IDEAM...")

    try:
        # Ir a la página principal
        url = "http://dhime.ideam.gov.co/atencionciudadano/"
        driver.get(url)
        print(f"✅ Página cargada: {url}")

        # Esperar un momento para que la página se cargue completamente
        time.sleep(3)

        # Manejar términos y condiciones
        if aceptar_terminos_condiciones(driver):
            print("✅ Acceso concedido al sistema DHIME")
            return True
        else:
            print("❌ No se pudo acceder al sistema DHIME")
            return False

    except Exception as e:
        print(f"❌ Error al navegar: {str(e)}")
        return False


In [47]:
# FUNCIÓN DE PRUEBA ESPECÍFICA PARA TU PROBLEMA
def test_acceso_ideam():
    """
    Función de prueba específica para verificar el acceso inicial al IDEAM
    """
    print("🧪 INICIANDO PRUEBA DE ACCESO AL IDEAM")
    print("=" * 50)

    driver = configurar_driver()

    try:
        if navegar_a_ideam(driver):
            print("🎉 ¡ÉXITO! Se pudo acceder correctamente al sistema DHIME")

            # Tomar screenshot para verificar
            driver.save_screenshot("screenshot_ideam_acceso.png")
            print("📸 Screenshot guardado como evidencia")

            # Esperar un poco para inspeccionar la página
            time.sleep(5)

            return True
        else:
            print("💥 FALLO: No se pudo acceder al sistema")
            driver.save_screenshot("screenshot_ideam_error.png")
            return False

    finally:
        driver.quit()

In [32]:
# EJEMPLO DE USO
if __name__ == "__main__":
    # Primero ejecutar la prueba de acceso
    print("EJECUTANDO PRUEBA DE ACCESO...")
    test_acceso_ideam()

EJECUTANDO PRUEBA DE ACCESO...
🧪 INICIANDO PRUEBA DE ACCESO AL IDEAM
🔧 Configurando el navegador Chrome...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Navegador configurado correctamente
🌐 Navegando al sitio del IDEAM...
✅ Página cargada: http://dhime.ideam.gov.co/atencionciudadano/
📋 Manejando términos y condiciones...
⏳ Esperando ventana de términos...
✅ Ventana de términos detectada
🔍 Intentando localizar checkbox - Estrategia 1...
✅ Checkbox marcado (Estrategia 1)
🔍 Buscando botón Aceptar...
❌ Botón por nombre no encontrado
❌ Botón por texto no encontrado
✅ Botón Aceptar clickeado (por clase dijit)
⏳ Esperando que se cierre la ventana...
✅ Términos y condiciones aceptados correctamente
✅ Acceso concedido al sistema DHIME
🎉 ¡ÉXITO! Se pudo acceder correctamente al sistema DHIME
📸 Screenshot guardado como evidencia
